# 1. Carregando as bibliotecas necessárias ao modelo

In [ ]:
%matplotlib inline

import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
# from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

from google.colab import drive
from IPython.display import display
from ipywidgets import FileUpload, widgets
from PIL import Image
import requests
from io import BytesIO

import warnings
warnings.filterwarnings("ignore")

# 2. Obtendo o conjunto de dados (dataset)



In [ ]:
import os
import zipfile
import urllib.request

# URL do dataset no GitHub
dataset_url = "https://github.com/jjofilho/transfer-learning-python/raw/main/dataset.zip"

# Nome do arquivo compactado
dataset_zip = "dataset.zip"

# Verifica se o dataset já está disponível
if not os.path.exists("dataset"):
    print("Baixando dataset...")
    urllib.request.urlretrieve(dataset_url, dataset_zip)

    # Extraindo o arquivo zip
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall(".")

    print("Dataset extraído com sucesso!")
else:
    print("Dataset já está disponível!")


# 3. Pré-processamento dos dados das imagens e o vetor entrada

In [ ]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

# 4. Carregando as imagens e identificando o número de classes para o modelo

In [ ]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames
              in os.walk(category) for f in filenames
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)
print(f'Número de classes no dataset: ', num_classes)

# 5. Definindo uma busca aleatória dos dados
Importante: o resultado apresentado do teste é alterado a cada execução da célula

In [ ]:
random.shuffle(data)

# 6. Criando: Treinamento | Validação | Teste (70%, 15%, 15%)

In [ ]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

# 7. Classificando os dados


In [ ]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

# 8. Pré-processamento dos dados como antes, certificando-se de que sejam float32 e normalizados entre 0 e 1

In [ ]:
# Normalizando os data
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Convertendo os labels para OneHotVector
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

# 9. Resumo do projeto até o momento

In [ ]:
# Resumo
print("Carregamento de %d imagens com %d categorias"%(len(data), num_classes))
print("Treinamento | Validação | Split Teste: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("Tamanho dos dados treinados: ", x_train.shape)
print("Tamanho dos labels: ", y_train.shape)


In [ ]:
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(root) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
idx = [int(len(images) * random.random()) for i in range(8)]
imgs = [image.load_img(images[i], target_size=(224, 224)) for i in idx]
concat_image = np.concatenate([np.asarray(img) for img in imgs], axis=1)
plt.figure(figsize=(16,4))
plt.imshow(concat_image)

# 10. Construção da rede neural base

In [ ]:
# Construindo a Rede Neural
model = Sequential()
print("Input dimensions: ",x_train.shape[1:])

model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

# 11. Treinando o modelo

In [ ]:
# Compilando o modelo para usar a função de perda 'categorical_crossentropy' e o otimizador 'adam' (AdaDelta)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_data=(x_val, y_val))


# 12. Avaliando a Perda de Validação e a Acurácia de Validação

In [ ]:
# Gráfico de precisão (accuracy)
fig = plt.figure(figsize=(12, 6))

# Loss (perda)
ax1 = fig.add_subplot(121)
ax1.plot(history.history["loss"], label="Treinamento")
ax1.plot(history.history["val_loss"], label="Validação")
ax1.set_title("Perda do Modelo")
ax1.set_xlabel("Época")
ax1.set_ylabel("Perda")
ax1.legend()
ax1.grid(True)

# Accuracy (acurácia)
ax2 = fig.add_subplot(122)
ax2.plot(history.history["accuracy"], label="Treinamento")
ax2.plot(history.history["val_accuracy"], label="Validação")
ax2.set_title("Acurácia do Modelo")
ax2.set_xlabel("Época")
ax2.set_ylabel("Acurácia")
ax2.legend()
ax2.grid(True)

plt.show()

# 13. Avaliação

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Teste de Perda:', loss)
print('Teste da Acurácia:', accuracy)

# 14. Transfer Learning começando com a rede existente

In [ ]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

In [ ]:
# camada de entrada do VGG
inp = vgg.input

# nova camada softmax com neurônios num_classes
new_classification_layer = Dense(num_classes, activation='softmax')

# conectando a nova camada à penúltima camada no VGG e faça uma referência a ela
out = new_classification_layer(vgg.layers[-4].output)

# criando a nova rede entre inp e out
model_new = Model(inp, out)

 # 15. Aplicando o Transfer Learning

In [ ]:
# Treinando  algumas camadas convolucionais para permitir ajuste fino e mantendo os pesos das demais fixas (congeladas)
for l, layer in enumerate(model_new.layers[:-4]):
    layer.trainable = False

for l, layer in enumerate(model_new.layers[-4:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

# 16. Conferindo o histórico do modelo após aplicado o Transfer Learning

In [ ]:
history2 = model_new.fit(x_train, y_train,
                         batch_size=128,
                         epochs=10,
                         validation_data=(x_val, y_val))


# 17. Apresentando o gráfico da Perda de Validação e Acurácia de Validação,
Modelo base, treinado do zero (azul) e o modelo aprendido por transferência (laranja-tracejado).

In [ ]:
# Gráfico com legendas e labels
fig = plt.figure(figsize=(16, 4))

# Gráfico da Perda de Validação
ax1 = fig.add_subplot(121)
ax1.plot(history.history["val_loss"], label="Modelo 1", color="blue", linestyle="-")
ax1.plot(history2.history["val_loss"], label="Modelo 2", color="orange", linestyle="--")
ax1.set_title("Perda de Validação", fontsize=14)
ax1.set_xlabel("Época", fontsize=12)
ax1.set_ylabel("Perda", fontsize=12)
ax1.legend(loc="upper right", fontsize=10)
ax1.grid(True)

# Gráfico da Acurácia de Validação
ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_accuracy"], label="Modelo 1", color="blue", linestyle="-")
ax2.plot(history2.history["val_accuracy"], label="Modelo 2", color="orange", linestyle="--")
ax2.set_title("Acurácia de Validação", fontsize=14)
ax2.set_xlabel("Época", fontsize=12)
ax2.set_ylabel("Acurácia", fontsize=12)
ax2.legend(loc="upper right", fontsize=10)
ax2.grid(True)

# 18. Nova validação no conjunto de teste

In [ ]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

# 19. Agora é sua vez de se divertir e aplicar o modelo!

A seguir, você pode carregar uma imagem válida, de violão ou guitarra, e aplicar o presente modelo de classificação com redes neurais.

Boa diversão!

In [ ]:
def classify_image_external(model, image_path, class_labels):
    """
    Classifica uma imagem utilizando o modelo treinado.

    Args:
        model: O modelo treinado para a classificação.
        image_path: Caminho ou objeto da imagem a ser classificada.
        class_labels: Lista com os nomes das classes (ex: ['violao', 'guitarra']).

    Returns:
        None. Exibe a imagem e a classe prevista.
    """
    # Carregar e preprocessar a imagem
    img, x = get_image(image_path)
    probabilities = model.predict([x])

    # Determinar a classe com maior probabilidade
    predicted_class = class_labels[np.argmax(probabilities)]

    # Exibir a imagem e o resultado da classificação
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Predição: {predicted_class}")
    plt.show()

    print(f"Probabilidade: {probabilities}")
    print(f"Classe predita: {predicted_class}")

# Lista de classes do modelo
class_labels = ["guitarra", "violão"]

# Função para processar imagem local
def upload_image(change):
    uploaded_file = next(iter(upload_widget.value.values()))
    content = uploaded_file['content']

    # Salvar a imagem para processamento
    with open("temp_image.jpg", "wb") as f:
        f.write(content)
    classify_image_external(model_new, "temp_image.jpg", class_labels)

# Função para processar imagem de URL
def classify_image_from_url(url):
    try:
        # Requisição HTTP para a URL
        response = requests.get(url)

        # Verificação se o conteúdo retornado é válido
        if response.status_code == 200 and "image" in response.headers["Content-Type"]:
            # Abrindo a imagem diretamente
            img = Image.open(BytesIO(response.content))
            img.save("temp_url_image.jpg")  # Salvar a imagem temporariamente
            classify_image_external(model_new, "temp_url_image.jpg", class_labels)
        else:
            print("Erro: O link fornecido não retorna uma imagem válida.")
    except Exception as e:
        print(f"Erro ao carregar a imagem da URL: {e}")

# Widgets para interação
upload_widget = FileUpload(accept='image/*', multiple=False)
url_widget = widgets.Text(
    value='',
    placeholder='Digite a URL da imagem',
    description='URL:',
    layout=widgets.Layout(width='50%')
)
button_url = widgets.Button(description="Classificar Imagem URL")
output = widgets.Output()

def on_button_url_click(b):
    with output:
        output.clear_output()
        classify_image_from_url(url_widget.value)

# Conectando os widgets às funções
upload_widget.observe(upload_image, names='value')
button_url.on_click(on_button_url_click)

# Exibir os widgets
display(widgets.VBox([
    widgets.HTML("<h3>Selecione uma Imagem:</h3>"),
    upload_widget,
    widgets.HTML("<h3>Ou forneça uma URL:</h3>"),
    url_widget,
    button_url,
    output
]))
